In [1]:
import tensorflow as tf
import numpy as np
import csv
import matplotlib
import matplotlib.pyplot as plt
import random
import cv2

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers import regression
from tflearn.data_utils import to_categorical


/home/van/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with open('hand_written.csv', 'r') as csv_file:
    result = csv.reader(csv_file)
    rows = []
    #đọc từng dòng file thêm vào rows
    # mỗi phần tử của rows là một dòng.
    # for row in result:
    #     rows.append(row)

    train_data = []
    train_label = []

    for letter in result:
        if (letter[0] == '0' or letter[0] == '1' or letter[0] == '2' or letter[0] == '3'):
            x = np.array([int(j) for j in letter[1:]])
            x = x.reshape(28, 28)
            train_data.append(x)
            train_label.append(int(letter[0]))
        else:
            break
print(len(train_label))
print(np.shape(train_label))


56081
(56081,)


In [3]:
#xáo trộn dữ liệu 
shuffer_order = list(range(56081))
random.shuffle(shuffer_order)

train_data = np.array(train_data)
train_label = np.array(train_label)

train_data = train_data[shuffer_order]
train_label = train_label[shuffer_order]

In [4]:
# chia tập dữ liệu ra training set, test set và validation set
train_x = train_data[:50000]
train_y = train_label[:50000]

val_x = train_data[50000:53000]  
val_y = train_label[50000:53000]

test_x = train_data[53000:]
test_y = train_label[53000:] 


In [5]:
img = cv2.imread('/home/van/Desktop/Project2/image/char2.jpg', 0)
print(img.shape)
img = cv2.resize(img, (28,28))
print(img.shape)
img = img.reshape((28*28))
img = img / 255.0
for i in range(len(img)):
    if img[i] < 0.5:
        img[i] = 0
    else:
        img[i] = 1
img = img.reshape((28,28))
cv2.imshow('image', img)
cv2.waitKey(6000)
cv2.destroyAllWindows()

(76, 76)
(28, 28)


In [6]:
test_x = np.array([img,])

In [7]:
BATCH_SIZE = 32
IMG_SIZE = 28
N_CLASSES = 4
LR = 0.001
N_EPOCHS = 50

In [8]:
tf.reset_default_graph()

network = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1]) #1

network = conv_2d(network, 32, 3, activation='relu') #2
network = max_pool_2d(network, 2) #3

network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)

network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)

network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)

network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)

network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)

network = fully_connected(network, 1024, activation='relu') #4
network = dropout(network, 0.8) #5

network = fully_connected(network, N_CLASSES, activation='softmax')#6
network = regression(network)

model = tflearn.DNN(network) #7

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [9]:
# đưa dữ liệu về định dạng phù hợp
train_x = train_x.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
val_x = val_x.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_x = test_x.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

original_test_y = test_y # được sử dụng để test ở bước sau
# chuyển label về dạng onhot vector 
train_y = to_categorical(train_y, N_CLASSES)
val_y = to_categorical(val_y, N_CLASSES)
test_y = to_categorical(test_y, N_CLASSES)

In [10]:
# training
model.fit(train_x, train_y, n_epoch=N_EPOCHS, validation_set=(val_x, val_y), show_metric=True)
model.save('model/mymodel.tflearn')

Training Step: 1075  | total loss: 0.02162 | time: 18.949s
| Adam | epoch: 002 | loss: 0.02162 - acc: 0.9924 -- iter: 18752/50000


KeyboardInterrupt: 

In [11]:
model.load('/home/van/Desktop/Project2/model/mymodel.tflearn')

INFO:tensorflow:Restoring parameters from /home/van/Desktop/Project2/model/mymodel.tflearn


In [12]:
test_logits = model.predict(test_x)

In [14]:
test_logits = np.argmax(test_logits, axis=-1)

In [15]:
print(test_logits)

[2]
